In [ ]:
import tensorflow as tf

a=tf.constant('helle')
with tf.Sess

In [ ]:
import os
from scipy import misc
import numpy as np
import tensorflow as tf
import math
import timeit

In [ ]:
data=np.loadtxt('34552_sci_OK_data(without noise).csv', delimiter=',', dtype=np.float32)
instances=data[:, 0:22500]
is_OK = data[:, 22500]
is_org = data[:,22501]
class_num = data[:,22502]

In [ ]:
aug_mask = np.where(is_org==0)
orig_mask = np.where(is_org==1)
orig_data = instances[orig_mask]
aug_data = instances[aug_mask]

print(orig_data.shape)
print(aug_data.shape)

In [ ]:
orig_is_OK = is_OK[orig_mask]
aug_is_OK = is_OK[aug_mask]
orig_OK_mask = np.where(orig_is_OK==1)
orig_NG_mask = np.where(orig_is_OK==0)
aug_OK_mask = np.where(aug_is_OK==1)
aug_NG_mask = np.where(aug_is_OK==0)
orig_OK_data = orig_data[orig_OK_mask]
orig_NG_data = orig_data[orig_NG_mask]
aug_OK_data = aug_data[aug_OK_mask]
aug_NG_data = aug_data[aug_NG_mask]

print(orig_OK_data.shape)
print(orig_NG_data.shape)
print(aug_OK_data.shape)
print(aug_NG_data.shape)

In [ ]:
orig_OK_data = orig_OK_data.reshape((-1,150,150,1))
orig_NG_data = orig_NG_data.reshape((-1,150,150,1))
aug_OK_data = aug_OK_data.reshape((-1,150,150,1))
aug_NG_data = aug_NG_data.reshape((-1,150,150,1))

print(orig_OK_data.shape)
print(orig_NG_data.shape)
print(aug_OK_data.shape)
print(aug_NG_data.shape)

In [ ]:
np.random.shuffle(orig_OK_data)
np.random.shuffle(orig_NG_data)
orig_OK_train = orig_OK_data[:13800]
orig_OK_test = orig_OK_data[13800:]
orig_NG_train = orig_NG_data[:2240]
orig_NG_test = orig_NG_data[2240:]

print(orig_OK_train.shape)
print(orig_OK_test.shape)
print(orig_NG_train.shape)
print(orig_NG_test.shape)

In [ ]:
orig_OK_train_label = np.ones(orig_OK_train.shape[0])
orig_NG_train_label = np.zeros(orig_NG_train.shape[0])
aug_NG_label = np.zeros(aug_NG_data.shape[0])
orig_OK_test_label = np.ones(orig_OK_test.shape[0])
orig_NG_test_label = np.zeros(orig_NG_test.shape[0])

In [ ]:
train_data = np.concatenate((orig_OK_train, orig_NG_train, aug_NG_data))
train_label = np.concatenate((orig_OK_train_label, orig_NG_train_label, aug_NG_label))
print(train_data.shape)
print(train_label.shape)

In [ ]:
test_data = np.concatenate((orig_OK_test, orig_NG_test))
test_label = np.concatenate((orig_OK_test_label, orig_NG_test_label))
print(test_data.shape)
print(test_label.shape)

In [ ]:
s = np.arange(train_data.shape[0])
np.random.shuffle(s)
train_data = train_data[s]
train_label = train_label[s]

s = np.arange(test_data.shape[0])
np.random.shuffle(s)
test_data = test_data[s]
test_label = test_label[s]

In [ ]:
val_data = train_data[24400:]
train_data = train_data[:24400]
val_label = train_label[24400:]
train_label = train_label[:24400]

In [ ]:
mean_image = np.mean(np.concatenate((val_data,train_data,test_data)),axis=0)
val_data -= mean_image
train_data -= mean_image
test_data -= mean_image
orig_OK_test -= mean_image
orig_NG_test -= mean_image

In [ ]:
del(instances)
del(is_OK)
del(is_org)
del(class_num)
del(aug_mask)
del(orig_mask)
del(orig_data)
del(aug_data)
del(orig_is_OK)
del(aug_is_OK)
del(orig_OK_mask)

In [ ]:
del(orig_NG_mask)
del(aug_OK_mask)
del(aug_NG_mask)
del(orig_OK_data)
del(orig_NG_data)
del(aug_OK_data)
del(aug_NG_data)
del(orig_OK_train)
#del(orig_OK_test)
del(orig_NG_train)
#del(orig_NG_test)
del(orig_OK_train_label)
del(orig_NG_train_label)
del(aug_NG_label)
#del(orig_OK_test_label)
#del(orig_NG_test_label)

In [ ]:
print(train_data.shape)
print(val_data.shape)
print(test_data.shape)
print(orig_OK_test.shape)
print(orig_NG_test.shape)

In [ ]:
def run_model(session, predict, loss_val, Xd, yd,
              epochs=1, batch_size=64, print_every=100,
              training=None, plot_losses=False):
    # have tensorflow compute accuracy
    correct_prediction = tf.equal(tf.argmax(predict,1), y)
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
    # shuffle indicies
    train_indicies = np.arange(Xd.shape[0])
    np.random.shuffle(train_indicies)

    training_now = training is not None
    
    # setting up variables we want to compute (and optimizing)
    # if we have a training function, add that to things we compute
    variables = [mean_loss,correct_prediction,accuracy]
    if training_now:
        variables[-1] = training
    
    # counter 
    iter_cnt = 0
    for e in range(epochs):
        # keep track of losses and accuracy
        correct = 0
        losses = []
        # make sure we iterate over the dataset once
        for i in range(int(math.ceil(Xd.shape[0]/batch_size))):
            # generate indicies for the batch
            start_idx = (i*batch_size)%Xd.shape[0]
            idx = train_indicies[start_idx:start_idx+batch_size]
            
            # create a feed dictionary for this batch
            feed_dict = {X: Xd[idx,:],
                         y: yd[idx],
                         is_training: training_now }
            # get batch size
            actual_batch_size = yd[idx].shape[0]
            # have tensorflow compute loss and correct predictions
            # and (if given) perform a training step
            loss, corr, _ = session.run(variables,feed_dict=feed_dict)
            
            # aggregate performance stats
            losses.append(loss*actual_batch_size)
            correct += np.sum(corr)
            
            # print every now and then
            if training_now and (iter_cnt % print_every) == 0:
                print("Iteration {0}: with minibatch training loss = {1:.3g} and accuracy of {2:.2g}"\
                      .format(iter_cnt,loss,np.sum(corr)/actual_batch_size))
            iter_cnt += 1
        total_correct = correct/Xd.shape[0]
        total_loss = np.sum(losses)/Xd.shape[0]
        print("Epoch {2}, Overall loss = {0:.3g} and accuracy of {1:.3g}"\
              .format(total_loss,total_correct,e+1))
        if plot_losses:
            plt.plot(losses)
            plt.grid(True)
            plt.title('Epoch {} Loss'.format(e+1))
            plt.xlabel('minibatch number')
            plt.ylabel('minibatch loss')
            plt.show()
    return total_loss,total_correct

In [ ]:
# Feel free to play with this cell

def my_model(X,y,is_training):
    Wconv1 = tf.Variable(tf.random_normal([3, 3, 1, 16],stddev=0.01))
    bconv1 = tf.Variable(tf.random_normal([16]))
    out = tf.nn.conv2d(X, Wconv1, strides=[1,1,1,1], padding='VALID') + bconv1
    out = tf.layers.batch_normalization(out, training=is_training)
    out = tf.nn.relu(out)
    out = tf.layers.max_pooling2d(inputs=out, pool_size=[2,2], padding="VALID", strides=2)
    #74
    
    Wconv2 = tf.Variable(tf.random_normal([3, 3, 16, 32],stddev=0.01))
    bconv2 = tf.Variable(tf.random_normal([32]))
    out = tf.nn.conv2d(out, Wconv2, strides=[1,1,1,1], padding='VALID') + bconv2
    out = tf.layers.batch_normalization(out, training=is_training)
    out = tf.nn.relu(out)  
    out = tf.layers.max_pooling2d(inputs=out, pool_size=[2,2], padding="VALID", strides=2)
    #36
    
    Wconv3 = tf.Variable(tf.random_normal([3, 3, 32, 32],stddev=0.01))
    bconv3 = tf.Variable(tf.random_normal([32]))
    out = tf.nn.conv2d(out, Wconv3, strides=[1,1,1,1], padding='VALID') + bconv3
    out = tf.layers.batch_normalization(out, training=is_training)
    out = tf.nn.relu(out)  
    out = tf.layers.max_pooling2d(inputs=out, pool_size=[2,2], padding="VALID", strides=2)
    #17
    
    flat_out = tf.reshape(out,shape=[-1,9248])
    
    W1 = tf.get_variable("W1", shape=[9248,64],initializer=tf.contrib.layers.xavier_initializer())
    b1 = tf.Variable(tf.random_normal([64]))
    out2 = tf.matmul(flat_out, W1) + b1
    out2 = tf.layers.batch_normalization(out2, training=is_training)
    out2 = tf.nn.relu(out2)
    #out2 = tf.layers.dropout(inputs=out2, rate=0.5, training=is_training)
    
    W2 = tf.get_variable("W2", shape=[64,2],initializer=tf.contrib.layers.xavier_initializer())
    b2 = tf.Variable(tf.random_normal([2]))
    out3 = tf.matmul(out2, W2) + b2
    
    return out3
    pass

tf.reset_default_graph()

X = tf.placeholder(tf.float32, [None, 150, 150, 1])
y = tf.placeholder(tf.int64, [None])
is_training = tf.placeholder(tf.bool)

y_out = my_model(X,y,is_training)
mean_loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y_out, labels=tf.one_hot(y,depth=2)))
optimizer = tf.train.AdamOptimizer(learning_rate=5e-3)


pass

# batch normalization in tensorflow requires this extra dependency
extra_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
with tf.control_dependencies(extra_update_ops):
    train_step = optimizer.minimize(mean_loss)

In [ ]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

for i in range(17):
    print(i)
    print('Training')
    run_model(sess,y_out,mean_loss,train_data,train_label,2,32,100,train_step,False)
    print('Validation')
    run_model(sess,y_out,mean_loss,val_data,val_label,1,64)

In [ ]:
print('Validation')
run_model(sess,y_out,mean_loss,val_data,val_label,1,64)

In [ ]:
print('Test: OK+NG')
run_model(sess,y_out,mean_loss,test_data,test_label,1,64)
print('Test: OK')
run_model(sess,y_out,mean_loss,orig_OK_test,orig_OK_test_label,1,64)
print('Test: NG')
run_model(sess,y_out,mean_loss,orig_NG_test,orig_NG_test_label,1,64)